# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do for any dataset is the pre-processing.  Implement the following pre-processing functions:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [3]:
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    # words = text.split()
    words = set(text) # "text" is a list of split word (not nevessary unique), where "words" is a set of unique words
    
    int_to_vocab = {_i:_w for _i, _w in enumerate(words)}
    vocab_to_int = {_w:_i for _i, _w in int_to_vocab.items()}
    
    # return tuple
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and token is the value:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** ) <-- This is actually a "new line" not "return", which is '\r'.

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token.
    """
    # TODO: Implement Function
    tokens_dict = dict()
    tokens_dict['.'] = '<PERIOD>'
    tokens_dict[','] = '<COMMA>'
    tokens_dict['"'] = '<QUATATION>'
    tokens_dict[';'] = '<SEMICOLON>'
    tokens_dict['!'] = '<EXCLAMATION>'
    tokens_dict['?'] = '<QUESTION>'
    tokens_dict['('] = '<L_PARENTHESES>'
    tokens_dict[')'] = '<R_PARENTHESES>'
    tokens_dict['-'] = '<DASH>'
    tokens_dict['\n'] = '<NEW_LINE>'
    return tokens_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to look at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [8]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    
#     # Method 1: The following implementation simply divides the whole dataset into chunks.
#     #           Where the variety in inputs and labels are limited and the utilization of the dataset will be quit low.
#     #           And because we will be randomly sample from these patches, there will be no meaningfull relation 
#     #           between batches; therefore, there will be no need to keep the hidden states when training.
#     #
#     #           TODO: Implement a method that can generate sample sequence start at any places.
#     #--------------------------------------------#
#     # Calculate the possible number of samples for the whole text. Reserve the last word as the label for the latest patch.
#     num_samples = (len(words) - 1) // sequence_length
    
#     # Generate a list of patches for input and label
#     patch_list = [words[_hi:(_hi+sequence_length)] for _hi in range(0, num_samples*sequence_length, sequence_length)] # (start, stop, step)
#     label_list = [words[_ei] for _ei in range(sequence_length, (num_samples*sequence_length+1), sequence_length)]
#     #--------------------------------------------#
    
    # Method 2: Generate patches form all possible start points
    #           Note: This method is inefficient in memory usage.
    #                 Because we will still shuffle the data, there is still no relationship between batches.
    #--------------------------------------------#
    word_length = len(words)
    patch_list = [words[_hi:(_hi+sequence_length)] for _hi in range(word_length-sequence_length-1)] # Reserve one for the label
    label_list = [words[_ei] for _ei in range(sequence_length, (word_length-1), 1)]
    #--------------------------------------------#
    
    
    # Convert to numpy arrays
    patch_list = np.array(patch_list)
    label_list = np.array(label_list)
    
    # Generate a PyTorch TensorDataset
    _dataset = TensorDataset(torch.from_numpy(patch_list), torch.from_numpy(label_list))
    
    # return a dataloader
    _data_loader = DataLoader(_dataset, shuffle=True, batch_size=batch_size)
    return _data_loader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [9]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 10,  11,  12,  13,  14],
        [ 20,  21,  22,  23,  24],
        [ 26,  27,  28,  29,  30],
        [ 41,  42,  43,  44,  45],
        [ 28,  29,  30,  31,  32],
        [ 25,  26,  27,  28,  29],
        [ 23,  24,  25,  26,  27],
        [  0,   1,   2,   3,   4],
        [  5,   6,   7,   8,   9],
        [  8,   9,  10,  11,  12]])

torch.Size([10])
tensor([ 15,  25,  31,  46,  33,  30,  28,   5,  10,  13])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [29]:
# Decide if the latest hidden states are passing from the current mini-batch to next mini-batch
# is_continuous_memory = True
is_continuous_memory = False # In this project, there are no relationship between review samples; therefore, it makes no sense to pass the hidden states.

In [30]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # TODO: Implement function
        
        # set class variables
        self.vocab_size  = vocab_size
        self.output_size = output_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        # define model layers
        self.embed = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = nn.GRU(self.embedding_dim, self.hidden_dim, num_layers=self.n_layers, batch_first=True, dropout=dropout)
        self.dropout = nn.Dropout(p=dropout)
        #
        self.fc = nn.Linear(self.hidden_dim, self.output_size) 
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function   
        batch_size = nn_input.size(0)
        
        # Embedding
        x = self.embed(nn_input)
        
        # GRU
        gru_out, hidden = self.gru(x, hidden[0]) 
        #                                   ^^^ 
        # Because the GRU got only one hidden state (different for LSTm),
        # to pass the unit test, I wrapped all the hidden state as a singleton 
        
        # GRU to output stage
        # "Flatten"
        y = gru_out.contiguous().view(-1, self.hidden_dim)

        # output stage
        y = self.dropout(y)
        y = self.fc(y)
        
        # Reshape back
        y = y.view(batch_size, -1, self.output_size)
        
        # Get the last time step
        y_last = y[:,-1] # <-- This equals to y[:,-1,:]
        
        # return one batch of output word scores and the hidden state
        return y_last, (hidden,) # <--To pass the unit test, I wrapped all the hidden state as a singleton 
    
    
    def init_hidden(self, batch_size, train_on_gpu=train_on_gpu):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        weight = next(self.parameters()).data
        
        # initialize hidden state with zero weights, and move to GPU if available
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()
        # Move to GPU if available
        if train_on_gpu:
            hidden = hidden.cuda()
        return (hidden,) # <--To pass the unit test, I wrapped all the hidden state as a singleton 

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [31]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden, is_continuous_memory=is_continuous_memory):
    """
    Forward and backward propagation on the neural network
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    
    # move data to GPU, if available
    if train_on_gpu:
        inp, target = inp.cuda(), target.cuda()
        
    # Block the trace for hidden
    _done_hidden_work = False
    if not is_continuous_memory:
        try:
            hidden = rnn.init_hidden(batch_size) # Reset the states
            _done_hidden_work = True
        except:
            pass
    if not _done_hidden_work:
        hidden = tuple([each.data for each in hidden]) # Keep the state's values but block the back-propegation
        
    
    # perform backpropagation and optimization
    output, hidden = rnn(inp, hidden)
    loss = criterion(output, target)
    rnn.zero_grad()
    loss.backward()
    # Clip
    clip = 5
    nn.utils.clip_grad_norm_(rnn.parameters(), clip)
    optimizer.step()

    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data being ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [32]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    filtered_training_loss = None
    
#     # Learning rate schedualling
#     reduce_factor = 0.5 # 0.2 # 0.5
#     # min_lr = 2.0*10**-6 # By experience, lower than this rate then there seems no progress anymore.
#     # min_lr = 1.0*10**-5 # 5*2*10**-6
#     min_lr = 6.2*10**-5 # By experience, lower than this rate then there seems no progress anymore.
#     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', 
#                                                            patience=(20*10), 
#                                                            factor=reduce_factor, 
#                                                            min_lr=min_lr, 
#                                                            verbose=True)
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):

        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)

        N_batches = len(train_loader)
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):

            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break

            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # Update filtered_training_loss
            _batch_avg_loss = np.average(batch_losses)
            if filtered_training_loss is None:
                filtered_training_loss = _batch_avg_loss
            else:
                filtered_training_loss += 0.02 * (_batch_avg_loss - filtered_training_loss) # 0.02

#             # Learning rate schedualing, monitor on filtered traning loss
#             scheduler.step(filtered_training_loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4} | Progress for this epoch:{}/{}={:.1f}% Loss: {} (avg: {})\n'.format(
                    epoch_i, n_epochs, batch_i, N_batches, (100*batch_i/float(N_batches)), 
                    _batch_avg_loss, filtered_training_loss))
                batch_losses = []


    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [33]:
# Data params
# Sequence Length
sequence_length = 10 # 10 # 20 --> loss:4.32  # of words in a sequence
# Batch Size
batch_size = 400 # 400 # 300 # 100

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [34]:
# Training parameters
# Number of Epochs
num_epochs = 5 # 10 # 3
# Learning Rate
learning_rate = 0.0005 # 0.001 # 0.0005 # 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = len(vocab_to_int)
# Embedding Dimension
embedding_dim = 1500 # 500 # 256
# Hidden Dimension
hidden_dim = 1000 # 1000 # 500
# Number of RNN Layers
n_layers = 1 # 2

# Show stats for every n number of batches
show_every_n_batches = 20 # 100 # 500

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [35]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
from workspace_utils import active_session

# Keep the connection
with active_session():
    # do long-running work here
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

/opt/conda/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Training for 5 epoch(s)...
Epoch:    1/5    | Progress for this epoch:20/2231=0.9% Loss: 7.260425782203674 (avg: 9.466843656638408)

Epoch:    1/5    | Progress for this epoch:40/2231=1.8% Loss: 5.462548995018006 (avg: 8.15771499019019)

Epoch:    1/5    | Progress for this epoch:60/2231=2.7% Loss: 5.133408069610596 (avg: 7.1773046304293295)

Epoch:    1/5    | Progress for this epoch:80/2231=3.6% Loss: 4.998850417137146 (avg: 6.453553191666149)

Epoch:    1/5    | Progress for this epoch:100/2231=4.5% Loss: 4.920315289497376 (avg: 5.967969210608781)

Epoch:    1/5    | Progress for this epoch:120/2231=5.4% Loss: 4.845413994789124 (avg: 5.601850271742378)

Epoch:    1/5    | Progress for this epoch:140/2231=6.3% Loss: 4.714849996566772 (avg: 5.315326391017718)

Epoch:    1/5    | Progress for this epoch:160/2231=7.2% Loss: 4.674214172363281 (avg: 5.095756316838047)

Epoch:    1/5    | Progress for this epoch:180/2231=8.1% Loss: 4.704705858230591 (avg: 4.961013354205681)

Epoch:    1/5 

Epoch:    1/5    | Progress for this epoch:1540/2231=69.0% Loss: 4.113836085796356 (avg: 4.152757561316534)

Epoch:    1/5    | Progress for this epoch:1560/2231=69.9% Loss: 4.110587704181671 (avg: 4.123235318267252)

Epoch:    1/5    | Progress for this epoch:1580/2231=70.8% Loss: 4.097427201271057 (avg: 4.119443960421839)

Epoch:    1/5    | Progress for this epoch:1600/2231=71.7% Loss: 4.232459831237793 (avg: 4.156428279716886)

Epoch:    1/5    | Progress for this epoch:1620/2231=72.6% Loss: 4.121806061267852 (avg: 4.157528771017383)

Epoch:    1/5    | Progress for this epoch:1640/2231=73.5% Loss: 4.144070649147034 (avg: 4.15887061816066)

Epoch:    1/5    | Progress for this epoch:1660/2231=74.4% Loss: 4.168801212310791 (avg: 4.168038336816918)

Epoch:    1/5    | Progress for this epoch:1680/2231=75.3% Loss: 4.14047064781189 (avg: 4.164979065848048)

Epoch:    1/5    | Progress for this epoch:1700/2231=76.2% Loss: 4.114867401123047 (avg: 4.14516851269638)

Epoch:    1/5    | Pro

Epoch:    2/5    | Progress for this epoch:840/2231=37.7% Loss: 3.7700112104415893 (avg: 3.799248706683787)

Epoch:    2/5    | Progress for this epoch:860/2231=38.5% Loss: 3.80911180973053 (avg: 3.8110937845294113)

Epoch:    2/5    | Progress for this epoch:880/2231=39.4% Loss: 3.860466814041138 (avg: 3.8042954838566)

Epoch:    2/5    | Progress for this epoch:900/2231=40.3% Loss: 3.767019498348236 (avg: 3.804434327679827)

Epoch:    2/5    | Progress for this epoch:920/2231=41.2% Loss: 3.857889008522034 (avg: 3.827197144880067)

Epoch:    2/5    | Progress for this epoch:940/2231=42.1% Loss: 3.833509111404419 (avg: 3.810666811335812)

Epoch:    2/5    | Progress for this epoch:960/2231=43.0% Loss: 3.865053343772888 (avg: 3.8139345819697663)

Epoch:    2/5    | Progress for this epoch:980/2231=43.9% Loss: 3.7794837474823 (avg: 3.811685223833692)

Epoch:    2/5    | Progress for this epoch:1000/2231=44.8% Loss: 3.8602787733078 (avg: 3.82855570767495)

Epoch:    2/5    | Progress for 

Epoch:    3/5    | Progress for this epoch:120/2231=5.4% Loss: 3.3983875036239626 (avg: 3.4483456367173595)

Epoch:    3/5    | Progress for this epoch:140/2231=6.3% Loss: 3.4453479051589966 (avg: 3.435257932849789)

Epoch:    3/5    | Progress for this epoch:160/2231=7.2% Loss: 3.4152509927749635 (avg: 3.419527603853969)

Epoch:    3/5    | Progress for this epoch:180/2231=8.1% Loss: 3.4531107187271117 (avg: 3.4228509256177766)

Epoch:    3/5    | Progress for this epoch:200/2231=9.0% Loss: 3.463779616355896 (avg: 3.4434845750863126)

Epoch:    3/5    | Progress for this epoch:220/2231=9.9% Loss: 3.4358909249305727 (avg: 3.444387573484919)

Epoch:    3/5    | Progress for this epoch:240/2231=10.8% Loss: 3.4677473664283753 (avg: 3.454961224880572)

Epoch:    3/5    | Progress for this epoch:260/2231=11.7% Loss: 3.489705944061279 (avg: 3.4684603890233405)

Epoch:    3/5    | Progress for this epoch:280/2231=12.6% Loss: 3.4345759391784667 (avg: 3.4577394264793555)

Epoch:    3/5    | Pro

...

Epoch:    4/5    | Progress for this epoch:920/2231=41.2% Loss: 3.18784955739975 (avg: 3.237299967749533)

Epoch:    4/5    | Progress for this epoch:940/2231=42.1% Loss: 3.231241297721863 (avg: 3.2441241557237164)

Epoch:    4/5    | Progress for this epoch:960/2231=43.0% Loss: 3.266631710529327 (avg: 3.251773211349866)

Epoch:    4/5    | Progress for this epoch:980/2231=43.9% Loss: 3.2913265347480776 (avg: 3.267636924469149)

Epoch:    4/5    | Progress for this epoch:1000/2231=44.8% Loss: 3.2857490301132204 (avg: 3.280659640813382)

Epoch:    4/5    | Progress for this epoch:1020/2231=45.7% Loss: 3.257081151008606 (avg: 3.273742897652961)

Epoch:    4/5    | Progress for this epoch:1040/2231=46.6% Loss: 3.279196929931641 (avg: 3.272283606918208)

Epoch:    4/5    | Progress for this epoch:1060/2231=47.5% Loss: 3.256470060348511 (avg: 3.2652638560806433)

Epoch:    4/5    | Progress for this epoch:1080/2231=48.4% Loss: 3.289979612827301 (avg: 3.2638984854051163)

Epoch:    4/5    | 

Epoch:    5/5    | Progress for this epoch:200/2231=9.0% Loss: 2.929221284389496 (avg: 2.90162463797257)

Epoch:    5/5    | Progress for this epoch:220/2231=9.9% Loss: 2.90474134683609 (avg: 2.8984051729556346)

Epoch:    5/5    | Progress for this epoch:240/2231=10.8% Loss: 2.876519334316254 (avg: 2.899383441840828)

Epoch:    5/5    | Progress for this epoch:260/2231=11.7% Loss: 2.9075789928436278 (avg: 2.9028447043849783)

Epoch:    5/5    | Progress for this epoch:280/2231=12.6% Loss: 2.953945243358612 (avg: 2.9171388851438858)

Epoch:    5/5    | Progress for this epoch:300/2231=13.4% Loss: 2.962524390220642 (avg: 2.933486493527374)

Epoch:    5/5    | Progress for this epoch:320/2231=14.3% Loss: 2.953367030620575 (avg: 2.933459500129616)

Epoch:    5/5    | Progress for this epoch:340/2231=15.2% Loss: 2.973357367515564 (avg: 2.9474363678710715)

Epoch:    5/5    | Progress for this epoch:360/2231=16.1% Loss: 2.929703450202942 (avg: 2.926373808849861)

Epoch:    5/5    | Progress

Epoch:    5/5    | Progress for this epoch:1720/2231=77.1% Loss: 3.099507141113281 (avg: 3.102032685193877)

Epoch:    5/5    | Progress for this epoch:1740/2231=78.0% Loss: 3.141036605834961 (avg: 3.112444870610797)

Epoch:    5/5    | Progress for this epoch:1760/2231=78.9% Loss: 3.1349148631095884 (avg: 3.1245970917997807)

Epoch:    5/5    | Progress for this epoch:1780/2231=79.8% Loss: 3.1200122833251953 (avg: 3.121825444019368)

Epoch:    5/5    | Progress for this epoch:1800/2231=80.7% Loss: 3.1341636419296264 (avg: 3.1189249506623327)

Epoch:    5/5    | Progress for this epoch:1820/2231=81.6% Loss: 3.1219656825065614 (avg: 3.1183309821721124)

Epoch:    5/5    | Progress for this epoch:1840/2231=82.5% Loss: 3.1920934200286863 (avg: 3.1445626769009887)

Epoch:    5/5    | Progress for this epoch:1860/2231=83.4% Loss: 3.1181085228919985 (avg: 3.1274651680884507)

Epoch:    5/5    | Progress for this epoch:1880/2231=84.3% Loss: 3.1685176372528074 (avg: 3.143025100701078)

Epoch: 

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved


In [36]:
# Save it here if the training was aboarted by me.
#----------------------------------------------------#
# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved


In [37]:
# Records
'''
Ver.1
---
sequence_length = 30
batch_size = 200
embedding_dim = 500 
hidden_dim = 256
n_layers = 3
---
Elbow: e:1/3 @ 23.9%, loss -> 4.6
'''

'''
Ver.2
---
sequence_length = 50 <-- Enlarge this seems not helping.
batch_size = 100
embedding_dim = 500 
hidden_dim = 500 <-- Almost no impact.
n_layers = 2
---
Elbow: e:1/3 @ 25.8%, loss -> 4.58
'''

'''
Ver.3
---
sequence_length = 100 <-- Enlarge this seems not helping.
batch_size = 100
embedding_dim = 256 <-- Shrinking this drops the performance a lot.
hidden_dim = 500
n_layers = 3
---
Elbow: e:1/3 @ 23.5%, loss -> 5.0
'''

'''
Ver.4
---
sequence_length = 20 <-- Shorter is better (for only considering one sentence at a time?)
batch_size = 300
embedding_dim = 500 <-- Larger is helping
hidden_dim = 500
n_layers = 2
---
Elbow: e:2/3 @ 8.1%, loss -> 4.09 
'''

'''
Ver.5
---
sequence_length = 20 
batch_size = 300
embedding_dim = 700 <-- Set even larger, no obvious use
hidden_dim = 500
n_layers = 2
---
Elbow: e:2/3 @ 9.4%, loss -> 4.2
'''

'''
Ver.6
---
sequence_length = 10 <-- try shorter sentence length
batch_size = 300
embedding_dim = 500 
hidden_dim = 500
n_layers = 2
---
Elbow: e:2/3 @ 2.0%, loss -> 4.2
'''

'''
Ver.7
---
sequence_length = 20 
batch_size = 300
embedding_dim = 500 
hidden_dim = 500
n_layers = 3 <-- Try deeper net
---
Elbow: e:1/3 @ 38.3%, loss -> 4.69
'''

'''
Ver.7
---
sequence_length = 15 <-- Shorter than 20
batch_size = 400 <-- Enlarge this for training faster
embedding_dim = 1000 <-- Even larger 
hidden_dim = 1000 <-- Grow with embedding_dim
n_layers = 2 
---
Elbow: e:2/5 @ 7.2%, loss -> 4.27
'''

'''
Ver.8 (same as 4, but disable continuous mode, i.e. is_continuous_memory=False)
---
is_continuous_memory = False <-- This helps,  a little bit.
sequence_length = 20 
batch_size = 300
embedding_dim = 500 
hidden_dim = 500
n_layers = 2
---
Elbow: e:2/5 @ 17.9%, avg_loss -> 4.103
'''

'''
Ver.9 (same as 8, but lower the learning rate)
---
is_continuous_memory = False 
sequence_length = 20 
batch_size = 300
embedding_dim = 500 
hidden_dim = 500
n_layers = 2
learning_rate = 0.0005 <-- Reduced from 0.001 to 0.0005
---
Elbow: e:2/10 @ 58.3%, avg_loss -> 4.015
'''

'''
Ver.10 (same as 9, but use a learning rate schaduling)
---
is_continuous_memory = False 
sequence_length = 20 
batch_size = 300
embedding_dim = 500 
hidden_dim = 500
n_layers = 2
learning_rate = 0.001 (reduce_factor 0.5) <-- Reduce 0.5x if stop progressing for 10*20 baches
---
lr: down to 1.9531e-06
Elbow: e:3/10 @ 18.8%, avg_loss -> 3.96
'''

'''
Ver.11 (same as 10, but tune the reduce_factor)
---
is_continuous_memory = False 
sequence_length = 20 
batch_size = 400
embedding_dim = 500 
hidden_dim = 500
n_layers = 2
learning_rate = 0.001 (reduce_factor 0.2) <-- Shrink this
---
lr: down to ??
Elbow: e:3/10 @ 0.9%, avg_loss -> 4.05
'''

'''
Ver.12 
---
is_continuous_memory = False 
sequence_length = 20 
batch_size = 400
embedding_dim = 1000 <-- Increase this 
hidden_dim = 1000 <-- Increase this
n_layers = 2
learning_rate = 0.001 (reduce_factor 0.5) <-- Backtrack to 0.5
---
lr: down to 1.9531e-06
Elbow: e:3/5 @ 4.5%, avg_loss -> 3.90
'''

'''
Ver.13
---
is_continuous_memory = False 
sequence_length = 20 
batch_size = 200
embedding_dim = 1200 <-- Increase this 
hidden_dim = 1200 <-- Increase this
n_layers = 2 
learning_rate = 0.001 (reduce_factor 0.5) 
---
lr: down to 1.5259e-08
Elbow: e:2/5 @ 11.7%, avg_loss -> 4.2
'''

'''
Ver.14 
---
is_continuous_memory = False 
sequence_length = 20 
batch_size = 400
embedding_dim = 1500 <-- Increase this
hidden_dim = 1000 <-- Fix this
n_layers = 2 
learning_rate = 0.001 (reduce_factor 0.5, min_lr=1*10^-5) 
---
lr: down to ??
Elbow: e:3/5 @ ??%, avg_loss -> 3.8
'''

'''
Ver.15
---
is_continuous_memory = False 
sequence_length = 10 <-- Decrease this 
batch_size = 400
embedding_dim = 1500 
hidden_dim = 1000    
n_layers = 2 
learning_rate = 0.001 (reduce_factor 0.5, min_lr=6.2*10^-5) 
---
lr: down to 6.2*10^-5
Elbow: e:4/5 @ 40.3%, avg_loss -> 3.795
'''

'''
Ver.15
---
is_continuous_memory = False 
sequence_length = 10 
batch_size = 400
embedding_dim = 3000  <-- Double this
hidden_dim = 1000    
n_layers = 2 
learning_rate = 0.001 (reduce_factor 0.5, min_lr=6.2*10^-5) 
---
lr: down to ??
Elbow: e:5/5 @ ??%, avg_loss -> 3.78
'''

'''
Ver.16
---
is_continuous_memory = False 
sequence_length = 10 
batch_size = 400
embedding_dim = 1500  <-- Shrink this
hidden_dim = 1000    
n_layers = 1 <-- Reduce this
learning_rate = 0.0005 (Fixed, disabled the learning rate scheduler) 
---
Elbow: e:5/5 @ 100%, avg_loss -> 3.16
'''

'\nVer.16\n---\nis_continuous_memory = False \nsequence_length = 10 \nbatch_size = 400\nembedding_dim = 1500  <-- Shrink this\nhidden_dim = 1000    \nn_layers = 1 <-- Reduce this\nlearning_rate = 0.0005 (disabled the learning rate scheduler) \n---\nlr: down to ??\nElbow: e:??/5 @ ??%, avg_loss -> ??\n'

### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** (Write answer, here)

After playing around for a while, I found that enlarging embedding_dim helps a lot, whereas tuning up hidden_dim along has less effect if embedding_dim was not tuned up accordingly. For sequence_length, I found it be less helpful, and the training process will become too slow when setting a relative large sequence_length, so I shrinked the sequence_length down to 10, which is about the length of a normal line of a character. Additionally, because the way I generate the data, a batch does not depend on its precedding batch. Therefore, I modified the training loop in forward_back_prop() so that it will re-initialize the hiddens for each batch rather than for each epoch. This helps more significantly when sequence_length is small. 

However, the loss didn't go below 3.5 until I shrinked the n_layers from 2 to 1. It seems like stacking multiple layers do not help in this case. 

The final version of the parameters are as following,

- is_continuous_memory = False 
- sequence_length = 10 
- batch_size = 400
- embedding_dim = 1500  
- hidden_dim = 1000    
- n_layers = 1 
- learning_rate = 0.0005 (Fixed) 

and the training loss reaches 3.16, meets the requirement.

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [38]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [39]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of punctuation tokens keys to punctuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [40]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:46: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


jerry: by insurance, you're making up a little nervous about a ride, huh?

jerry: yeah?

elaine: well, i don't have to tell you, it's a little secret.

elaine: well what do you think of your mind? you want a muffin to do that?

elaine: no.

jerry: what do you mean? you know how to do that i was in the neighborhood, but i think i saw her in a position.

george: what, you think i would do something like that?

elaine:(confused) what?

newman: you know, i think i have a problem with the velvet fog and the saleswoman.

george: what are you doing?

jerry: i'm not a good mood.

elaine:(still looking at the picture) oh, that's my new plates!(to george) you know, i think i was a kid, he could have a pony..

elaine: i think i can see you in my office.

kramer:(still laughing) oh, i don't think so.

kramer: yeah..(to elaine) you have no idea!

elaine: no, no i don't.

jerry:(trying to keep them) oh, that's nice.

jerry:(pointing at her) oh, hi.

kramer: hi.(jerry looks at the watch and then re- 

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [41]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.